In [1]:
# Connect to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unrar x "/content/drive/MyDrive/20Crypto.rar" "content/"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/20Crypto.rar

Creating    content                                                   OK
Extracting  content/data_XRPUSDT.pkl                                       6%  OK 
Extracting  content/data_ARKUSDT.pkl                                       9%  OK 
Extracting  content/data_BTCUSDT.pkl                                      17% 18%  OK 
Extracting  content/data_DUSDT.pkl                                        22%  OK 
Extracting  content/data_FORMUSDT.pkl                                     26%  OK 
Extracting  content/data_GMTUSDT.pkl                                      31%  OK 
Extracting  content/data_HBARUSDT.pkl                                     38%  OK 
Extracting  content/data_HOTUSDT.pkl                                      42%  OK 
Extracting  content/data_JSTUSDT.pkl                  

Loading Embeded news based on Finbert

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df_news = pd.read_pickle('/content/drive/MyDrive/news_20crypto_embed_finbert.pickle')

Embedding No News sentence for using in timestamp with no news

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load FinBERT model and tokenizer
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Your input news text
text = "No news."

# Tokenize and encode the text
inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)

# Get the model output
with torch.no_grad():
    outputs = model(**inputs)

# Use the [CLS] token embedding as sentence embedding
cls_embedding = outputs.last_hidden_state[:, 0, :]  # Shape: [1, 768]

# Convert to numpy array if needed
embedding_vector = cls_embedding.squeeze().numpy()

print("Embedding shape:", embedding_vector.shape)
# print("Embedding vector:", embedding_vector)
no_news_vector = list(embedding_vector)

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Embedding shape: (768,)


Loading OHLCV of multi stocks

In [5]:
import os

list_crypto = [x[5:-4] for x in os.listdir('/content/content/') if 'data_' in x and 'FORMUSDT' not in x]
# list_crypto = list_crypto[4:10]

In [6]:
list_data = []
selected_f_asset = ['open', 'high', 'low', 'close', 'volume',
    'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol',
       'takerBuyQuoteVol']
selected_f_asset = ['close', 'volume', 'numberOfTrades', 'return']
selected_f_all = []
list_target = []

for symb in list_crypto:
  df = pd.read_pickle('/content/content/data_{}.pkl'.format(symb))
  print(symb, len(df))
  df['return'] = 100*df['close'].pct_change()
  df = df[selected_f_asset].rename(columns={x:symb+'_'+x for x in selected_f_asset})
  selected_f_all.extend(list(df.columns))
  df[symb+'_target'] = df[symb+'_'+'return'].rolling(30).std().shift(-30)
  list_target.append(symb+'_target')
  list_data.append(df.copy())

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

BTCUSDT 131041
XLMUSDT 131041
XRPUSDT 131041
TSTUSDT 131041
WUSDT 131041
ARKUSDT 131041
MEUSDT 131041
TUSDT 131041
OGUSDT 131041
TIAUSDT 131041
TRXUSDT 131041
SUSDT 131041
GMTUSDT 131041
JSTUSDT 131041
SEIUSDT 131041
HBARUSDT 131041
DUSDT 131041
USDCUSDT 131041
HOTUSDT 131041


In [7]:
len(selected_f_all)

76

In [8]:
data = pd.concat(list_data,axis=1)
df = pd.read_pickle('/content/content/data_{}.pkl'.format('BTCUSDT'))
df['return'] = df['close'].pct_change()
data['dateTime'] = df['dateTime']
data['time'] = data['dateTime'].astype(str).copy()
data['return'] = df['close'].pct_change()
data

,BTCUSDT_close,BTCUSDT_volume,BTCUSDT_numberOfTrades,BTCUSDT_return,BTCUSDT_target,XLMUSDT_close,XLMUSDT_volume,XLMUSDT_numberOfTrades,XLMUSDT_return,XLMUSDT_target,...,USDCUSDT_return,USDCUSDT_target,HOTUSDT_close,HOTUSDT_volume,HOTUSDT_numberOfTrades,HOTUSDT_return,HOTUSDT_target,dateTime,time,return
0,91785.18,44.25582000,9784,NaN,0.177998,0.2989,112334.00000000,392,NaN,0.422107,...,NaN,0.006948,0.001424,3866530.00000000,22,NaN,0.468796,2025-02-25 00:00:00,2025-02-25 00:00:00,NaN
1,91837.75,29.64367000,7048,0.057275,0.186473,0.2990,209664.00000000,398,0.033456,0.439614,...,0.000000,0.007184,0.001425,1135592.00000000,21,0.070225,0.474921,2025-02-25 00:01:00,2025-02-25 00:01:00,0.000573
2,91960.97,70.22032000,6386,0.134171,0.184519,0.3005,80701.00000000,311,0.501672,0.431379,...,0.000000,0.007427,0.001425,96487.00000000,4,0.000000,0.475486,2025-02-25 00:02:00,2025-02-25 00:02:00,0.001342
3,91862.77,50.83532000,4539,-0.106784,0.184458,0.3000,174073.00000000,666,-0.166389,0.434661,...,0.000000,0.007427,0.001431,1772593.00000000,14,0.421053,0.470420,2025-02-25 00:03:00,2025-02-25 00:03:00,-0.001068
4,91670.09,62.93148000,11779,-0.209748,0.182009,0.2988,101822.00000000,546,-0.400000,0.428329,...,-0.009999,0.007184,0.001420,484068.00000000,9,-0.768693,0.451829,2025-02-25 00:04:00,2025-02-25 00:04:00,-0.002097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131036,109394.14,1.80238000,514,-0.006490,NaN,0.2857,1341.00000000,5,0.000000,NaN,...,0.000000,NaN,0.001111,7873.00000000,1,-0.089928,NaN,2025-05-26 23:56:00,2025-05-26 23:56:00,-0.000065
131037,109409.09,5.64688000,698,0.013666,NaN,0.2857,11422.00000000,21,0.000000,NaN,...,-0.010004,NaN,0.001113,1793903.00000000,64,0.180018,NaN,2025-05-26 23:57:00,2025-05-26 23:57:00,0.000137
131038,109417.56,0.52540000,224,0.007742,NaN,0.2858,402.00000000,3,0.035002,NaN,...,0.010005,NaN,0.001113,1397911.00000000,9,0.000000,NaN,2025-05-26 23:58:00,2025-05-26 23:58:00,0.000077
131039,109434.79,3.88664000,1349,0.015747,NaN,0.2858,4370.00000000,33,0.000000,NaN,...,-0.010004,NaN,0.001115,1792102.00000000,10,0.179695,NaN,2025-05-26 23:59:00,2025-05-26 23:59:00,0.000157


Preparing embedding of news (Handling multi news and no news)

In [9]:
import numpy as np
df_news['embedding'] = df_news['embedding'].apply(lambda x: np.array(x))

df_grouped = df_news[['time', 'embedding', 'count']].groupby('time')['embedding'].apply(
    lambda x: np.mean(np.stack(x.values), axis=0)
).reset_index()
data_all = data.merge(df_grouped, on='time', how='left')
data_all['embedding'] = data_all['embedding'].apply(
    lambda x: no_news_vector if x is np.nan or x is None else x
)

In [10]:
list_crypto_first = [x[:-4] for x in list_crypto]
list_crypto_first

['BTC',
 'XLM',
 'XRP',
 'TST',
 'W',
 'ARK',
 'ME',
 'T',
 'OG',
 'TIA',
 'TRX',
 'S',
 'GMT',
 'JST',
 'SEI',
 'HBAR',
 'D',
 'USDC',
 'HOT']

In [11]:
data_all = data_all[data_all['time']>=df_news['time'].min()][data_all['time']<=df_news['time'].max()]

Showing Data

In [12]:
data_all[selected_f_all]

,BTCUSDT_close,BTCUSDT_volume,BTCUSDT_numberOfTrades,BTCUSDT_return,XLMUSDT_close,XLMUSDT_volume,XLMUSDT_numberOfTrades,XLMUSDT_return,XRPUSDT_close,XRPUSDT_volume,...,DUSDT_numberOfTrades,DUSDT_return,USDCUSDT_close,USDCUSDT_volume,USDCUSDT_numberOfTrades,USDCUSDT_return,HOTUSDT_close,HOTUSDT_volume,HOTUSDT_numberOfTrades,HOTUSDT_return
0,91785.18,44.25582000,9784,NaN,0.2989,112334.00000000,392,NaN,2.2986,809446.00000000,...,65,NaN,1.0001,874701.00000000,924,NaN,0.001424,3866530.00000000,22,NaN
1,91837.75,29.64367000,7048,0.057275,0.2990,209664.00000000,398,0.033456,2.3041,517300.00000000,...,9,-0.130225,1.0001,1386578.00000000,1009,0.000000,0.001425,1135592.00000000,21,0.070225
2,91960.97,70.22032000,6386,0.134171,0.3005,80701.00000000,311,0.501672,2.3147,229433.00000000,...,32,0.443343,1.0001,1136218.00000000,716,0.000000,0.001425,96487.00000000,4,0.000000
3,91862.77,50.83532000,4539,-0.106784,0.3000,174073.00000000,666,-0.166389,2.3108,288404.00000000,...,61,-0.038946,1.0001,740184.00000000,645,0.000000,0.001431,1772593.00000000,14,0.421053
4,91670.09,62.93148000,11779,-0.209748,0.2988,101822.00000000,546,-0.400000,2.2979,668514.00000000,...,6,-0.259740,1.0000,2203330.00000000,592,-0.009999,0.001420,484068.00000000,9,-0.768693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129596,109030.02,21.06713000,3921,0.130437,0.2878,12764.00000000,41,0.104348,2.3434,99206.80000000,...,1,-0.072202,0.9995,1145036.00000000,215,0.000000,0.001126,0.00000000,0,0.000000
129597,109024.39,5.10152000,1789,-0.005164,0.2877,26887.00000000,36,-0.034746,2.3432,87402.20000000,...,1,0.120424,0.9995,623068.00000000,119,0.000000,0.001126,0.00000000,0,0.000000
129598,108995.95,7.48873000,769,-0.026086,0.2878,21890.00000000,44,0.034758,2.3417,34994.50000000,...,4,-0.216502,0.9995,144815.00000000,146,0.000000,0.001126,0.00000000,0,0.000000
129599,109004.19,6.56241000,1298,0.007560,0.2878,22099.00000000,25,0.000000,2.3417,31291.00000000,...,0,0.000000,0.9995,17396.00000000,90,0.000000,0.001126,123675.00000000,1,0.000000


Normalizing Data Z-Score

In [13]:
##Normalization

for x in selected_f_all:
  data_all[x] = data_all[x].fillna(0)
  data_all[x] = data_all[x].astype(float)
  data_all[x] = (data_all[x]-data_all[x].mean())/(data_all[x].std())

Create time index for TimseNet

In [14]:
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader

df_stamp = data_all[['time']].rename(columns={'time':'dateTime'})
df_stamp['dateTime'] = pd.to_datetime(df_stamp['dateTime'])
df_stamp['month'] = df_stamp.dateTime.apply(lambda row: row.month, 1)
df_stamp['day'] = df_stamp.dateTime.apply(lambda row: row.day, 1)
df_stamp['weekday'] = df_stamp.dateTime.apply(lambda row: row.weekday(), 1)
df_stamp['hour'] = df_stamp.dateTime.apply(lambda row: row.hour, 1)
df_stamp['minute'] = df_stamp.dateTime.apply(lambda row: row.minute, 1)
data_stamp = df_stamp.drop(['dateTime'], axis=1)

Dataset Class and Creating DataLoader

In [15]:
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NewsTimeSeriesDataset(Dataset):
    def __init__(self, df, data_stamp, selected_f_all, stock_list, seq_len=30):
        self.df = df.reset_index(drop=True)
        self.data_stamp = data_stamp.reset_index(drop=True)
        self.features = selected_f_all
        self.seq_len = seq_len
        self.stock_list = [x+'_target' for x in stock_list]

    def __len__(self):
        return len(self.df) - self.seq_len

    def __getitem__(self, idx):
        ts_window = self.df.loc[idx:idx + self.seq_len - 1, self.features].values.astype('float32')
        x_mask = self.data_stamp.loc[idx:idx + self.seq_len - 1].values.astype('float32')
        news_window = self.df.loc[idx:idx + self.seq_len - 1, 'embedding'].values
        news_window = np.stack(news_window).astype('float32')
        target = self.df.loc[idx + self.seq_len, self.stock_list].values.astype('float32')  # shape: [19]
        return {
            'timeseries': torch.tensor(ts_window),     # [30, 114]
            'news': torch.tensor(news_window),         # [30, 768]
            'target': torch.tensor(target),
            'time_mask':torch.tensor(x_mask)# [19]
        }


split = int(0.8 * len(data_all))

train_dataset = NewsTimeSeriesDataset(data_all.iloc[:split], data_stamp.iloc[:split], selected_f_all, list_crypto, seq_len=30)
val_dataset = NewsTimeSeriesDataset(data_all.iloc[split:], data_stamp.iloc[split:], selected_f_all, list_crypto, seq_len=30)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


Model Architecture

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import math

import torch
import torch.nn as nn


class Inception_Block_V1(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V1, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=2 * i + 1, padding=i))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res


class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()


class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()
        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(
            self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x


class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6,
                    'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x) + self.position_embedding(x)
        else:
            x = self.value_embedding(
                x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)


class DataEmbedding_inverted(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding_inverted, self).__init__()
        self.value_embedding = nn.Linear(c_in, d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        x = x.permute(0, 2, 1)
        # x: [Batch Variate Time]
        if x_mark is None:
            x = self.value_embedding(x)
        else:
            x = self.value_embedding(torch.cat([x, x_mark.permute(0, 2, 1)], 1))
        # x: [Batch Variate d_model]
        return self.dropout(x)


class DataEmbedding_wo_pos(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding_wo_pos, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x)
        else:
            x = self.value_embedding(x) + self.temporal_embedding(x_mark)
        return self.dropout(x)


class PatchEmbedding(nn.Module):
    def __init__(self, d_model, patch_len, stride, padding, dropout):
        super(PatchEmbedding, self).__init__()
        # Patching
        self.patch_len = patch_len
        self.stride = stride
        self.padding_patch_layer = nn.ReplicationPad1d((0, padding))

        # Backbone, Input encoding: projection of feature vectors onto a d-dim vector space
        self.value_embedding = nn.Linear(patch_len, d_model, bias=False)

        # Positional embedding
        self.position_embedding = PositionalEmbedding(d_model)

        # Residual dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # do patching
        n_vars = x.shape[1]
        x = self.padding_patch_layer(x)
        x = x.unfold(dimension=-1, size=self.patch_len, step=self.stride)
        x = torch.reshape(x, (x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
        # Input encoding
        x = self.value_embedding(x) + self.position_embedding(x)
        return self.dropout(x), n_vars

from torch import Tensor
class GatedCrossAttentionFusion2D(nn.Module):
    """
    Fusion module for MSGCA with 2D inputs:
    1. Cross-attention between two modalities.
    2. Gated feature selection guided by a primary modality.
    Inputs are 2D: (batch_size, d_model)
    """
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        # Cross-attention projections (for queries, keys, values)
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)

        # Gating mechanism projections
        self.gate_proj = nn.Linear(d_model, d_model)  # For primary modality
        self.unstable_proj = nn.Linear(d_model, d_model)  # For unstable features

        # Output projection
        self.out_proj = nn.Linear(d_model, d_model)

    def forward(self,
                primary: torch.Tensor,
                auxiliary: torch.Tensor,
                ) -> torch.Tensor:
        """
        Args:
            primary:   (batch_size, d_model)
            auxiliary: (batch_size, d_model)
        Returns:
            fused:     (batch_size, d_model)
        """
        # --- Step 1: Unstable Cross-Attention Fusion ---
        Q = self.q_proj(primary)  # (batch, d_model)
        K = self.k_proj(auxiliary)
        V = self.v_proj(auxiliary)

        # Multi-head attention for 2D inputs
        batch_size, _ = Q.size()

        # Reshape for multi-head attention
        # (batch, n_heads, head_dim)
        Q = Q.view(batch_size, self.n_heads, self.head_dim)
        K = K.view(batch_size, self.n_heads, self.head_dim)
        V = V.view(batch_size, self.n_heads, self.head_dim)

        # Scaled dot-product attention
        # (batch, n_heads, n_heads)
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)
        # (batch, n_heads, head_dim)
        attn_output = torch.matmul(attn_weights, V)

        # Restore original shape: (batch, d_model)
        attn_output = attn_output.view(batch_size, self.d_model)

        # --- Step 2: Stable Gated Feature Selection ---
        # (batch, d_model)
        unstable_features = self.unstable_proj(attn_output)
        gate = torch.sigmoid(self.gate_proj(primary))

        # Element-wise gating
        # (batch, d_model)
        fused = unstable_features * gate

        # Output projection
        fused = self.out_proj(fused)
        return fused

class MSGCAFusion(nn.Module):
    """
    Complete fusion module for MSGCA:
    1. Fuses indicators + documents.
    2. Fuses (indicators + documents) + graph.
    """
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        # First fusion: Indicators (primary) + Documents (auxiliary)
        self.fusion1 = GatedCrossAttentionFusion2D(d_model, n_heads)


    def forward(self,
                indicators: Tensor,  # (batch, seq_len, d_model)
                documents: Tensor  # (batch, seq_len, d_model)
                ) -> Tensor:
        # First fusion stage
        fused_id = self.fusion1(primary=indicators, auxiliary=documents)  # (batch, seq_len, d_model)

        return fused_id





In [17]:
class InceptionBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.1):
        super().__init__()
        self.branch1 = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 7), padding=(0, 3))  # Extended time horizon for short-term patterns
        self.branch2 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 5), padding=(1, 2))  # Balanced time-feature relationships
        self.branch3 = nn.Conv2d(in_channels, out_channels, kernel_size=(7, 3), padding=(3, 1))  # Enhanced cross-feature interactions
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(out_channels * 3)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out = torch.cat([out1, out2, out3], dim=1)
        out = self.relu(self.bn(out))
        return self.dropout(out)

class TimesBlock(nn.Module):
    def __init__(self, d_model, top_k=3, dropout=0.1):
        super().__init__()
        self.top_k = top_k
        self.d_model = d_model
        self.inception = InceptionBlock2D(1, 1, dropout)
        self.output_proj = nn.Linear(3, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, C, T = x.shape
        freq = torch.fft.rfft(x, dim=-1)
        amplitude = freq.abs()[:, :, 1:]
        topk_indices = torch.topk(amplitude, self.top_k, dim=-1)[1]
        periods = (T // (topk_indices + 1)).clamp(min=2)
        output = torch.zeros_like(x)

        for k in range(self.top_k):
            p = periods[:, :, k]
            median_p = int(torch.median(p).item())
            seq_len = (T // median_p) * median_p
            views = x[:, :, :seq_len].reshape(B, C, -1, median_p)
            views = views.view(B * C, 1, views.shape[2], median_p)
            conv_out = self.inception(views)
            conv_out = conv_out.mean(dim=[2, 3])
            conv_out = conv_out.view(B, C, -1)
            proj_out = self.output_proj(conv_out)
            output += proj_out

        return self.dropout(output / self.top_k) + x


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft
from transformers import AutoTokenizer, AutoModel
finbert_model_name = "yiyanghkust/finbert-tone"



def FFT_for_Period(x, k=2):
    # [B, T, C]
    xf = torch.fft.rfft(x, dim=1)
    # find period by amplitudes
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0
    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list
    return period, abs(xf).mean(-1)[:, top_list]


class TimesBlock(nn.Module):
    def __init__(self, configs):
        super(TimesBlock, self).__init__()
        self.seq_len = configs['seq_len']
        self.pred_len = configs['pred_len']
        self.k = configs['top_k']
        # parameter-efficient design
        self.conv = nn.Sequential(
            Inception_Block_V1(configs['d_model'], configs['d_ff'],
                               num_kernels=configs['num_kernels']),
            nn.GELU(),
            Inception_Block_V1(configs['d_ff'], configs['d_model'],
                               num_kernels=configs['num_kernels'])
        )

    def forward(self, x):
        B, T, N = x.size()
        period_list, period_weight = FFT_for_Period(x, self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if (self.seq_len + self.pred_len) % period != 0:
                length = (
                                 ((self.seq_len + self.pred_len) // period) + 1) * period
                padding = torch.zeros([x.shape[0], (length - (self.seq_len + self.pred_len)), x.shape[2]]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = (self.seq_len + self.pred_len)
                out = x
            # reshape
            out = out.reshape(B, length // period, period,
                              N).permute(0, 3, 1, 2).contiguous()
            # 2D conv: from 1d Variation to 2d Variation
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            res.append(out[:, :(self.seq_len + self.pred_len), :])
        res = torch.stack(res, dim=-1)
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=1)
        period_weight = period_weight.unsqueeze(
            1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res


class Model_TimesNet(nn.Module):
    """
    Paper link: https://openreview.net/pdf?id=ju_Uqw384Oq
    """

    def __init__(self, configs):
        super(Model_TimesNet, self).__init__()
        self.configs = configs
        self.seq_len = configs['seq_len']
        self.label_len = configs['label_len']
        self.pred_len = configs['pred_len']
        self.finbert = AutoModel.from_pretrained(finbert_model_name)
        self.finbert_proj = nn.Linear(self.finbert.config.hidden_size, configs['d_model'])
        self.model = nn.ModuleList([TimesBlock(configs)
                                    for _ in range(configs['e_layers'])])
        self.enc_embedding = DataEmbedding(configs['enc_in'], configs['d_model'], configs['embed'], configs['freq'],
                                           configs['dropout'])
        self.layer = configs['e_layers']
        self.layer_norm = nn.LayerNorm(configs['d_model'])
        self.predict_linear = nn.Linear(
            self.seq_len, self.pred_len + self.seq_len)
        self.projection = nn.Linear(
            configs['d_model'], configs['c_out'], bias=True)
        self.projection2 = nn.Linear(
            configs['c_out'], configs['num_class'], bias=True)
        self.fusion = MSGCAFusion(configs['d_model'], n_heads=8)


    def forecast(self, x_enc, x_mark_enc, input_ids, attention_mask):
        # embedding
        enc_out = self.enc_embedding(x_enc, x_mark_enc)  # [B,T,C]
        enc_out = self.predict_linear(enc_out.permute(0, 2, 1)).permute(
            0, 2, 1)  # align temporal dimension
        # TimesNet
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        # porject back

        enc_out = enc_out[:, -self.pred_len:, :].squeeze(1)

        with torch.set_grad_enabled(self.training):
            outputs = self.finbert(input_ids=input_ids, attention_mask=attention_mask)
            cls_embed = outputs.last_hidden_state[:, 0, :]  # [B, hidden]
        x_news = self.finbert_proj(cls_embed)               # [B, d_model]

        fusion = self.fusion(enc_out, x_news)
        # fusion = torch.cat([enc_out, x_news], dim=-1)      # [B, d_model*2]
        dec_out = self.projection(fusion)
        return dec_out

    def forward(self, x_enc, x_mark_enc, input_ids, attention_mask):
        dec_out = self.forecast(x_enc, x_mark_enc, input_ids, attention_mask)

        return dec_out  # [B, L, D]


import torch.nn as nn

class SharedFusionModel(nn.Module):
    def __init__(self, configs, ts_input_dim, news_embed_dim, hidden_dim=64, num_stocks=19):
        super().__init__()
        self.num_stocks = num_stocks
        self.configs = configs
        self.seq_len = configs['seq_len']
        self.label_len = configs['label_len']
        self.pred_len = configs['pred_len']
        self.model = nn.ModuleList([TimesBlock(configs)
                                    for _ in range(configs['e_layers'])])
        self.enc_embedding = DataEmbedding(configs['enc_in'], configs['d_model'], configs['embed'], configs['freq'],
                                           configs['dropout'])
        self.layer = configs['e_layers']
        self.layer_norm = nn.LayerNorm(configs['d_model'])
        self.predict_linear = nn.Linear(
            self.seq_len, self.pred_len + self.seq_len)
        self.projection = nn.Linear(
            configs['d_model'], configs['c_out'], bias=True)
        self.projection2 = nn.Linear(
            configs['c_out'], configs['num_class'], bias=True)
        self.fusion = MSGCAFusion(configs['d_model'], n_heads=4)


        # 1. Project input to Transformer-compatible dim
        self.ts_proj = nn.Linear(ts_input_dim, 128)
        self.ts_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=128, nhead=4, dim_feedforward=256, batch_first=True),
            num_layers=2
        )
        self.ts_out = nn.Linear(128, hidden_dim)

        # 2. News LSTM
        self.news_proj = nn.Linear(news_embed_dim, 64)
        self.news_lstm = nn.LSTM(input_size=64, hidden_size=hidden_dim, batch_first=True)

        # 3. Stock-specific regression heads (1 per stock)
        self.stock_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            )
            for _ in range(num_stocks)
        ])

    def forward(self, ts_input, x_mark_enc, news_input):  # [B, 30, F], [B, 30, E]


        enc_out = self.enc_embedding(ts_input, x_mark_enc)  # [B,T,C]
        enc_out = self.predict_linear(enc_out.permute(0, 2, 1)).permute(
            0, 2, 1)  # align temporal dimension
        # TimesNet
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        # porject back

        ts_emb = enc_out[:, -self.pred_len:, :].squeeze(1)


        # ts_encoded = self.ts_proj(ts_input)
        # ts_out = self.ts_encoder(ts_encoded)           # [B, 30, 128]
        # ts_emb = self.ts_out(ts_out[:, -1, :])
        # print(ts_emb.shape)


                # [B, 64]
        news_proj = self.news_proj(news_input)         # [B, 30, 64]
        _, (hn, _) = self.news_lstm(news_proj)        # hn: [1, B, 64]
        news_emb = hn[-1]                              # [B, 64]

        # print(ts_emb.shape)
        # print(news_emb.shape)

        fused = self.fusion(ts_emb, news_emb)
        # fused = torch.cat([ts_emb, news_emb], dim=1)   # [B, 128]

        outputs = [head(fused).squeeze(-1) for head in self.stock_heads]
        return torch.stack(outputs, dim=1)             # [B, 19]

class SharedFusionModel(nn.Module):
    def __init__(self, configs, ts_input_dim, news_embed_dim, hidden_dim=64, num_stocks=19):
        super().__init__()
        self.num_stocks = num_stocks
        self.configs = configs
        self.seq_len = configs['seq_len']
        self.label_len = configs['label_len']
        self.pred_len = configs['pred_len']
        self.num_stocks = num_stocks
        self.model = nn.ModuleList([TimesBlock(configs)
                                    for _ in range(configs['e_layers'])])
        self.enc_embedding = DataEmbedding(configs['enc_in'], configs['d_model'], configs['embed'], configs['freq'],
                                           configs['dropout'])
        self.layer = configs['e_layers']
        self.layer_norm = nn.LayerNorm(configs['d_model'])
        self.predict_linear = nn.Linear(
            self.seq_len, self.pred_len + self.seq_len)
        self.projection = nn.Linear(
            configs['d_model'], configs['c_out'], bias=True)
        self.projection2 = nn.Linear(
            configs['c_out'], configs['num_class'], bias=True)
        # self.fusion = MSGCAFusion(configs['d_model'], n_heads=4)
        self.fusion = nn.ModuleList([MSGCAFusion(configs['d_model'], n_heads=4) for _ in range(num_stocks)])

        # 1. Project input to Transformer-compatible dim
        self.ts_proj = nn.Linear(ts_input_dim, 128)
        self.ts_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=128, nhead=4, dim_feedforward=256, batch_first=True),
            num_layers=2
        )
        self.ts_out = nn.Linear(128, hidden_dim)

        # 2. News LSTM
        self.news_proj = nn.Linear(news_embed_dim, 64)
        self.news_lstm = nn.LSTM(input_size=64, hidden_size=hidden_dim, batch_first=True)

        # 3. Stock-specific regression heads (1 per stock)
        self.stock_heads = nn.ModuleList([
            nn.Sequential(
                nn.ReLU(),
                nn.Linear(64, 1)
            )
            for _ in range(self.num_stocks)
        ])

    def forward(self, ts_input, x_mark_enc, news_input):  # [B, 30, F], [B, 30, E]


        enc_out = self.enc_embedding(ts_input, x_mark_enc)  # [B,T,C]
        enc_out = self.predict_linear(enc_out.permute(0, 2, 1)).permute(
            0, 2, 1)  # align temporal dimension
        # TimesNet
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        # porject back

        ts_emb = enc_out[:, -self.pred_len:, :].squeeze(1)


        # ts_encoded = self.ts_proj(ts_input)
        # ts_out = self.ts_encoder(ts_encoded)           # [B, 30, 128]
        # ts_emb = self.ts_out(ts_out[:, -1, :])
        # print(ts_emb.shape)


                # [B, 64]
        news_proj = self.news_proj(news_input)         # [B, 30, 64]
        _, (hn, _) = self.news_lstm(news_proj)        # hn: [1, B, 64]
        news_emb = hn[-1]                              # [B, 64]

        # print(ts_emb.shape)
        # print(news_emb.shape)

        fused = [fusion(ts_emb, news_emb) for fusion in self.fusion]
        # fused = torch.cat([ts_emb, news_emb], dim=1)   # [B, 128]

        outputs = [self.stock_heads[ij](fused[ij]).squeeze(-1) for ij in range(self.num_stocks)]

        return torch.stack(outputs, dim=1)


seq_len = 30
# Configs
configs = {
    'task_name': 'classification',
    'seq_len': seq_len,
    'enc_in': len(selected_f_all),
    'd_model': 64,
    'c_out':2, #len(selected_f),
    'd_ff': 128,
    'num_kernels': 3,
    'dropout': 0.1,
    'e_layers': 2,
    'top_k': 3,
    'num_class': 2,
    'label_len':30,
    'pred_len':1,
    'embed':'timeF',
    'freq':'t'
}
# model = TimesNet.Model(args).float()
# model = Model_TimesNet(configs).float()

In [18]:
data_all = data_all.fillna(0)

In [19]:
 MSGCAFusion(16, 4)

MSGCAFusion(
  (fusion1): GatedCrossAttentionFusion2D(
    (q_proj): Linear(in_features=16, out_features=16, bias=True)
    (k_proj): Linear(in_features=16, out_features=16, bias=True)
    (v_proj): Linear(in_features=16, out_features=16, bias=True)
    (gate_proj): Linear(in_features=16, out_features=16, bias=True)
    (unstable_proj): Linear(in_features=16, out_features=16, bias=True)
    (out_proj): Linear(in_features=16, out_features=16, bias=True)
  )
)

Loading Model and dataloader

In [20]:
from torch.utils.data import DataLoader

# Example setup
# selected_f_all = list of feature columns (e.g., 6 × 19 = 114)
# stock_list = ['AAPL', 'GOOG', ..., 'TSLA'] → 19 stock target columns
# df = your dataframe with those columns and 'embedding' column


split = int(0.8 * len(data_all))

train_dataset = NewsTimeSeriesDataset(data_all.iloc[:split], data_stamp.iloc[:split], selected_f_all, list_crypto, seq_len=30)
val_dataset = NewsTimeSeriesDataset(data_all.iloc[split:], data_stamp.iloc[split:], selected_f_all, list_crypto, seq_len=30)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


model = SharedFusionModel(
    configs,
    ts_input_dim=len(selected_f_all),
    news_embed_dim=len(data_all['embedding'].iloc[0]),
    hidden_dim=64,
    num_stocks=len(list_crypto)
).cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()



In [21]:
len(data_all['embedding'].iloc[0])

768

Testing model pipeline

In [ ]:
# One training step
for batch in train_loader:
    ts_input = batch['timeseries'].cuda()
    news_input = batch['news'].cuda()
    target = batch['target'].cuda()
    time_mask = batch['time_mask'].cuda()

    output = model(ts_input, time_mask, news_input)  # [B, 19]
    loss = loss_fn(output, target)
    break

Training model

In [ ]:
from sklearn.metrics import r2_score
from tqdm import tqdm
import torch
import numpy as np
import os

def compute_r2_per_stock(y_true, y_pred):
    r2s = []
    for i in range(y_true.shape[1]):
        r2s.append(r2_score(y_true[:, i], y_pred[:, i]))
    return r2s

num_epochs=50
patience=5
save_path='best_model.pth'
device='cuda'

best_val_loss = float('inf')
patience_counter = 0
model.to(device)

for epoch in range(1, num_epochs + 1):
    model.train()
    total_train_loss = 0

    print(f"\n🔁 Epoch {epoch}/{num_epochs}")
    for step, batch in enumerate(tqdm(train_loader, desc="Training")):
        ts_input = batch['timeseries'].to(device)
        news_input = batch['news'].to(device)
        target = batch['target'].to(device)
        time_mask = batch['time_mask'].to(device)

        output = model(ts_input, time_mask, news_input)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        # Step-wise print every 100 steps
        if step % 100 == 0:
            avg_loss = total_train_loss / (step + 1)
            print(f"  Step {step}/{len(train_loader)} - Batch Loss: {loss.item():.4f} - Avg Loss: {avg_loss:.4f}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    all_preds, all_targets = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            ts_input = batch['timeseries'].to(device)
            news_input = batch['news'].to(device)
            target = batch['target'].to(device)
            time_mask = batch['time_mask'].to(device)

            output = model(ts_input, time_mask, news_input)
            loss = loss_fn(output, target)

            total_val_loss += loss.item()
            all_preds.append(output.cpu().numpy())
            all_targets.append(target.cpu().numpy())

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    y_pred = np.concatenate(all_preds, axis=0)
    y_true = np.concatenate(all_targets, axis=0)
    r2_scores = compute_r2_per_stock(y_true, y_pred)

    print(f"📊 Epoch {epoch} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss: {avg_val_loss:.4f}")
    # print("  R² per stock:", ["{:.3f}".format(r) for r in r2_scores])
    print("  R² per stock:", ["{} , {:.3f}".format(list_crypto_first[r], r2_scores[r]) for r in range(len(r2_scores))])

    if epoch % 5 == 0:
        np.save(f"y_pred_epoch_{epoch}.npy", y_pred)
        np.save(f"y_true_epoch_{epoch}.npy", y_true)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved.")
    else:
        patience_counter += 1
        print(f"⏳ Early stopping patience: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("⛔ Early stopping triggered.")
        break



🔁 Epoch 1/50


Training:   0%|          | 1/3240 [00:03<3:14:38,  3.61s/it]

  Step 0/3240 - Batch Loss: 0.0340 - Avg Loss: 0.0340


Training:   3%|▎         | 101/3240 [01:12<37:10,  1.41it/s]

  Step 100/3240 - Batch Loss: 0.0041 - Avg Loss: 0.0142


...

Training:   0%|          | 1/3240 [00:00<33:17,  1.62it/s]

  Step 0/3240 - Batch Loss: 0.0129 - Avg Loss: 0.0129


Training:   3%|▎         | 101/3240 [01:03<32:45,  1.60it/s]

  Step 100/3240 - Batch Loss: 0.0017 - Avg Loss: 0.0033


Training:   4%|▎         | 117/3240 [01:13<33:05,  1.57it/s]

In [ ]:
print("R² per stock: ", ["{:.3f}".format(r2) for r2 in r2_scores])

R² per stock:  ['0.054', '-1.257', '0.308', '0.229', '0.011', '-0.009', '0.272', '0.196', '-4.720', '0.313', '-0.141', '0.072', '-7.790', '-0.279', '-0.023', '-0.274', '-0.394', '-0.097', '0.343']
